In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np

In [2]:
overlay = Overlay("design_1_wrapper.bit")

In [19]:
overlay?

In [ ]:
overlay.

In [4]:
# access to the hierarchy and dma
#hier = overlay.moving_average_hier
dma = overlay.axi_dma_0

In [5]:
dim = 1000
window = 3

In [6]:
data_in = np.random.randint(-1000, 1000, dim, np.int32)
#data_in = np.empty(dim, np.int32)
#for i in range(dim):
    #data_in[i]=i

In [7]:
data_in[0:10]

array([   8, -416, -862,  258, -291,  -50,  861,  -26,  848,  605],
      dtype=int32)

In [8]:
# moving average software function
def moving_average(data, window):
    data_out_size = data.shape[0] - window + 1
    data_out = np.empty(data_out_size, np.float32)
    for i in range(data_out_size):
        data_out[i] = np.sum(data[i:i + window])/window
    return data_out 

In [9]:
data_out = moving_average(data_in, window)

In [10]:
# create hardware buffers
buf_in = allocate(dim + 2, np.int32)
buf_out = allocate(dim - window + 1, np.float32)

In [11]:
# fill input buffer
buf_in[0] = dim
buf_in[1] = window
buf_in[2:] = data_in[:]

In [12]:
print(data_in[0:10])
print(buf_in[0:10])

[   8 -416 -862  258 -291  -50  861  -26  848  605]
[1000    3    8 -416 -862  258 -291  -50  861  -26]


In [13]:
# send and receive data
dma.recvchannel.transfer(buf_out)
dma.sendchannel.transfer(buf_in)
# wait for data
dma.sendchannel.wait()
dma.recvchannel.wait()

In [14]:
buf_out[0:10]

PynqBuffer([-423.33334 , -340.      , -298.33334 ,  -27.666666,
             173.33333 ,  261.66666 ,  561.      ,  475.66666 ,
             686.      ,  678.      ], dtype=float32)

In [15]:
data_out[0:20]

array([-423.33334 , -340.      , -298.33334 ,  -27.666666,  173.33333 ,
        261.66666 ,  561.      ,  475.66666 ,  686.      ,  678.      ,
        185.33333 , -337.66666 , -697.3333  , -454.66666 , -160.      ,
        223.33333 ,  269.33334 ,  305.      ,  316.66666 ,  139.      ],
      dtype=float32)

In [16]:
# check results
print(np.all(buf_out == data_out))

True


In [17]:
buf_out.invalidate()

In [18]:
# run experiments on window values from 1 to 16
max_window = 16
for i in range(1, max_window + 1):
    print(i)
    data_out = moving_average(data_in, i)
    buf_in = allocate(dim + 2, np.int32)
    buf_out = allocate(dim - i + 1, np.float32)
    buf_in[0] = dim
    buf_in[1] = i
    buf_in[2:] = data_in[:]
    buf_in.flush()
    
    dma.recvchannel.transfer(buf_out)
    dma.sendchannel.transfer(buf_in)

    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    buf_out.invalidate()
    
    print(np.all(buf_out == data_out))

1
True
2
True
3
True
4
True
5
True
6
True
7
True
8
True
9
True
10
True
11
True
12
True
13
True
14
True
15
True
16
True
